# Step 1: Install the Qdrant Client
To interact with Qdrant, we need the Python client. This enables us to communicate with the Qdrant service, manage collections, and perform vector searches.

In [1]:
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 7.9 MB/s eta 0:00:00


# Step 2: Import Required Libraries
Import the necessary modules from the qdrant-client package. The QdrantClient class establishes connection to Qdrant, while the models module provides configurations for Distance, VectorParams, and PointStruct.

In [2]:
from qdrant_client import QdrantClient, models

# Step 3: Connect to Qdrant Cloud
To connect to Qdrant Cloud, you need your cluster URL and API key from your Qdrant Cloud dashboard. Replace with your actual credentials

In [6]:
#import os
#client = QdrantClient(url=os.getenv("CLUSTER_ENDPOINT"), api_key=os.getenv("QDRANT_API"))

# For Colab:
from google.colab import userdata
client = QdrantClient(url=userdata.get("CLUSTER_ENDPOINT"), api_key=userdata.get("QDRANT_API"))

# Step 4: Create a Collection
A collection in Qdrant is like a table in relational databases - a container for storing vectors and their metadata. When creating a collection, specify:

- Name: A unique identifier for the collection
- Vector Configuration:
    - Size: The dimensionality of the vectors
    - Distance Metric: The method to measure similarity between vectors

In [7]:
# Define the collection name
collection_name = "my_first_collection"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=4,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

# Step 5: Verify Collection Creation

In [8]:
# Retrieve and display the list of collections
collections = client.get_collections()
print("Existing collections:", collections)

Existing collections: collections=[CollectionDescription(name='my_first_collection')]


# Step 6: Insert Points into the Collection
Points are the core data entities in Qdrant. Each point contains:

- ID: A unique identifier
- Vector Data: An array of numerical values representing the data point in vector space
- Payload (Optional): Additional metadata

In [9]:
# Define the vectors to be inserted
points = [
    models.PointStruct(
        id=1,
        vector=[0.1, 0.2, 0.3, 0.4],  # 4D vector
        payload={"category": "example"}  # Metadata (optional)
    ),
    models.PointStruct(
        id=2,
        vector=[0.2, 0.3, 0.4, 0.5],
        payload={"category": "demo"}
    )
]

# Insert vectors into the collection
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=1, status=<UpdateStatus.COMPLETED: 'completed'>)

# Step 7: Retrieve Collection Details
Now that we’ve inserted vectors, let’s confirm they’re stored correctly by getting collection information:

In [10]:
collection_info = client.get_collection(collection_name)
print("Collection info:", collection_info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> warnings=None indexed_vectors_count=0 points_count=2 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=4, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None, inline_storage=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=10000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead

# Step 8: Run Your First Similarity Search
Find the most similar vector to a given query using Qdrant’s search capabilities:

How Similarity Search Works:

- Qdrant searches the collection to find the vectors that are closest to your query vector.
- The results are ranked by their similarity score, with the best matches appearing first.

In [11]:
query_vector = [0.08, 0.14, 0.33, 0.28]

search_results = client.query_points(
    collection_name=collection_name,
    query=query_vector,
    limit=1  # Return the top 1 most similar vector
)

print("Search results:", search_results)

Search results: points=[ScoredPoint(id=1, version=1, score=0.97642946, payload={'category': 'example'}, vector=None, shard_key=None, order_value=None)]
